# Example system of active PD equations

Say I have 3 substrates that are degraded by 2 substrates. 

1. $\( k_{\text{deg1}} = \frac{k_{\text{cat1}} \cdot [P_1]}{K_{m1} + [S_1]} \)$

2. $\( k_{\text{deg2}} = \frac{k_{\text{cat2}} \cdot [P_2]}{K_{m2} + [S_2]} \)$

3. $\( k_{\text{deg3}} = \frac{k_{\text{cat3}} \cdot [P_1]}{K_{m3} + [S_3]} + \frac{k_{\text{cat4}} \cdot [P_2]}{K_{m4} + [S_3]} \)$

4. $\( k_{\text{P1}} = k_{\text{cat1}} + k_{\text{cat3}} \)$

5. $\( k_{\text{P2}} = k_{\text{cat2}} + k_{\text{cat4}} \)$

By rearranging equations 4 and 5 to get $\( k_{\text{cat3}} \)$ and $\( k_{\text{cat4}} \)$ in terms of $\( k_{\text{cat1}} \)$ and $\( k_{\text{cat2}} \)$, respectively, we can substitute them into equation 3 to get:

6. $\( k_{\text{deg3}} = \frac{(k_{\text{P1}} - k_{\text{cat1}}) \cdot [P_1]}{K_{m3}) + [S_3]} + \frac{(k_{\text{P2}} - k_{\text{cat2}}) \cdot [P_2]}{K_{m4} + [S_3]} \)$

Then, we can rearrange equations 1 and 2 to get $\( k_{c1} \)$ and $\( k_{c2} \)$ in terms of $\( k_{\text{deg1}} \)$ and $\( k_{\text{deg2}} \)$, respectively, givening the following: 

$\( k_{\text{cat1}} = \frac{k_{\text{deg1}} \cdot K_{m1}}{[P_1]} \)$, 

$\( k_{\text{cat2}} = \frac{k_{\text{deg2}} \cdot K_{m2}}{[P_2]} \)$

Now I can plug in the values of $\( k_{\text{cat1}} \)$ and $\( k_{\text{cat2}} \)$ into equation 6 and then use numerical methods to solve for solutions to the $\( K_m \)$ values (since Kms appear in the denominator, I cannot use linear algebra): 


$\( k_{\text{deg3}} = \frac{(k_{\text{P1}} - \frac{k_{\text{deg2}} \cdot K_{m2}}{[P_2]}) \cdot [P_1]}{K_{m3}) + [S_3]} + \frac{(k_{\text{P2}} - \frac{k_{\text{deg2}} \cdot K_{m2}}{[P_2]}) \cdot [P_2]}{K_{m4} + [S_3]} \)$






In some cases, the relative contribution of a protease to the degradation of a subrate degraded by multiple substrates is given. In cases, like that, we can use that fraction to possibly simplify the system of equations by multiplying the contribution fraction by the rate of degradation of the substrate and setting that quantity equatl to the rate of degradation of the substrate by the particular protease.  

NameError: name 'cell_paths' is not defined

In [4]:
from scipy.optimize import fsolve
import numpy as np

Attempt to solve the equations above with the following three proteins: 

At steady state in our model, Lon (P1) has a value of log10(protein counts) = 2.854175 and clpP (P2) has a value of log10(protein counts) = 9.096198

metR: assinged to the Lon protease (P1), and has log10(protein counts) = 0.257029, and a half life of 48.7 minutes.

glpD: assigned to the clpP protease (P2), and has log10(protein counts) = 3.946707, and a half life of 210.9 minutes.

ibpA: assigned to lon and clpP, and has log10(protein counts) = 2.057573, and a half life of 72 minutes.

For now, we will assume that the kcat totals for each protease are equal to values found in literature (although these will be estimated better for the context of our model in the future). I will also assume Km3 = Km4 for now, and again, this will later be replaced with a deep curation of the values available in literature.

## Constants

In [20]:
# relative protein counts at steady state : 
P1 = 10**2.854175
P2 = 10**9.096198
S1 = 10**0.257029
S2 = 10**3.946707
S3 = 10**2.057573
# todo: do I need to turn this into the molar mass and then divide by the volume of the cell to get the concentration?
# convert to concentrations: 
# todo: use the ecocyc table to fine concentration values for thse complexes (bc Clpp def is one!!!) 
P1 = P1 
P2 = P2 
S1 = S1 
S2 = S2 
S3 = S3 

# measured degradation contstants: 
kdeg1 = np.log(2) / 48.7 # units: mins-1
kdeg2 = np.log(2) / 210.9 # units: mins-1
kdeg3 = np.log(2) / 72 # units: mins-1

k_total_p1 = 3.2 # s-1, value from https://www.sciencedirect.com/science/article/pii/S0003269701949887?via%3Dihub
k_total_p2 = 0.3 # min-1, enz-1, micromolar-1, value from: https://www.cell.com/cell-reports/pdf/S2211-1247(22)01242-6.pdf (enz-1 means per enzyme, so I believe this is ok)

# todo: figure out what to make the micromolar vale convert to!! do i need all the values in my above equations to have micromolar taken into account? 

# edit the values of the protease concentrations: 
k_total_p1 = k_total_p1 / 60 # convert from s-1 to mins-1
k_total_p2 = k_total_p2 # todo: figure out what to do conversion wise here! 


In [24]:
# Define system of equations
def equations(variables):
    kcat1, kcat2, Km1, Km2, Km3, Km4 = variables

    # Express kcat3 and kcat4 in terms of total kcat values
    kcat3 = k_total_p1 - kcat1
    kcat4 = k_total_p2 - kcat2

    # Equation system: kdeg1, kdeg2, kdeg3 equations, equations from total kcat definitions, and assume Km3 = Km4
    eq1 = kdeg1 - (kcat1 * P1 / (Km1 + S1))
    eq2 = kdeg2 - (kcat2 * P2 / (Km2 + S2))
    eq3 = kdeg3 - ((kcat3 * P1 / (Km3 + S3)) + (kcat4 * P2 / (Km4 + S2)))
    eq4 = kcat1 + kcat3 - k_total_p1  
    eq5 = kcat2 + kcat4 - k_total_p2  
    eq6 = Km4 - Km2 

    return [eq1, eq2, eq3, eq4, eq5, eq6]

# Initial guess for kcat1, kcat2, Km1, Km2, Km3, Km4
initial_guess = [0.1, 0.1, 0.5, 1, 0.5, 0.5]

# Use fsolve to solve the system of equations
solution = fsolve(equations, initial_guess)

kcat1, kcat2, Km1, Km2, Km3, Km4 = solution

print(f"Solved values:")
print(f"kcat1 = {kcat1}")
print(f"kcat2 = {kcat2}")
print(f"Km1 = {Km1}")
print(f"Km2 = {Km2}")
print(f"Km3 = {Km3}")
print(f"Km4 = {Km4}")

Solved values:
kcat1 = 2517305732.637409
kcat2 = 0.1504363803274422
Km1 = 52080635478.80615
Km2 = 878503.9208155256
Km3 = 1537221725243130.0
Km4 = 878503.9002423795
